In [16]:
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Embedding
from keras.layers import LSTM
from keras.layers import Conv1D ,MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [17]:
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [18]:
import ast

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
import tensorflow_addons as tfa

In [20]:
seed = 0
np.random.seed(seed)
tf.random.set_seed(3)

In [21]:
data = pd.read_csv("FINAL_RE_PLC_review_tokenized_okt_30377.csv")

In [22]:
data['review_tokens'] = data['review_tokens'].apply(lambda x: ast.literal_eval(x))

In [23]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['review_tokens'])
X = tokenizer.texts_to_sequences(data['review_tokens']) # Sequence 변환
max_len = 82
X = pad_sequences(X, max_len)
le = LabelEncoder()
y = le.fit_transform(data.plc)
y = to_categorical(y)

In [24]:
vocab_size = len(tokenizer.word_index) + 2

In [25]:
embedding_matrix = np.load('embedding_mat.npy')

## LSTM + CNN 구현

In [26]:
from tensorflow.keras.layers import Dense, Embedding, Bidirectional, GRU, Concatenate, Dropout, SpatialDropout1D
from tensorflow.keras import Input, Model
from tensorflow.keras import optimizers
import os

In [27]:
model = Sequential()
model.add(Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=max_len, trainable=False))
model.add(SpatialDropout1D(rate = 0.2))
model.add(Conv1D(filters = 64, kernel_size = 5, padding = 'same', activation = 'relu', strides = 1)) # padding = 'valid'
model.add(MaxPooling1D())
model.add(Bidirectional(LSTM(128)))
model.add(Dense(units = 300, activation='relu'))
model.add(Dropout(rate = 0.2))  
model.add(Dense(349, activation='softmax'))
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=[tfa.metrics.FBetaScore(num_classes=349, average='micro', beta=0.5)])

In [28]:
early_stopping = EarlyStopping(monitor='loss', patience=5)
val_f1_score = float('-inf')
model_checkpoint = ModelCheckpoint('CNN_BiLSTM.h5', monitor = 'fbeta_score', mode = 'max', verbose = 1, save_best_only = True)

In [29]:
history = model.fit(X, y, epochs=150, callbacks=[early_stopping, model_checkpoint], batch_size=128)

Epoch 1/150
238/238 [==============================] - 69s 226ms/step - loss: 4.9101 - fbeta_score: 0.0562

Epoch 00001: fbeta_score improved from -inf to 0.06130, saving model to CNN_BiLSTM.h5
Epoch 2/150
238/238 [==============================] - 54s 227ms/step - loss: 4.4402 - fbeta_score: 0.1067

Epoch 00002: fbeta_score improved from 0.06130 to 0.12572, saving model to CNN_BiLSTM.h5
Epoch 3/150
238/238 [==============================] - 53s 221ms/step - loss: 3.9377 - fbeta_score: 0.1753

Epoch 00003: fbeta_score improved from 0.12572 to 0.19265, saving model to CNN_BiLSTM.h5
Epoch 4/150
238/238 [==============================] - 53s 224ms/step - loss: 3.6231 - fbeta_score: 0.2269

Epoch 00004: fbeta_score improved from 0.19265 to 0.23399, saving model to CNN_BiLSTM.h5
Epoch 5/150
238/238 [==============================] - 53s 223ms/step - loss: 3.4049 - fbeta_score: 0.2607

Epoch 00005: fbeta_score improved from 0.23399 to 0.26635, saving model to CNN_BiLSTM.h5
Epoch 6/150
238/23

238/238 [==============================] - 55s 230ms/step - loss: 1.5340 - fbeta_score: 0.5710

Epoch 00042: fbeta_score improved from 0.55884 to 0.56131, saving model to CNN_BiLSTM.h5
Epoch 43/150
238/238 [==============================] - 55s 230ms/step - loss: 1.5329 - fbeta_score: 0.5706

Epoch 00043: fbeta_score improved from 0.56131 to 0.56539, saving model to CNN_BiLSTM.h5
Epoch 44/150
238/238 [==============================] - 55s 229ms/step - loss: 1.5082 - fbeta_score: 0.5779

Epoch 00044: fbeta_score improved from 0.56539 to 0.57096, saving model to CNN_BiLSTM.h5
Epoch 45/150
238/238 [==============================] - 55s 230ms/step - loss: 1.4841 - fbeta_score: 0.5839

Epoch 00045: fbeta_score improved from 0.57096 to 0.57346, saving model to CNN_BiLSTM.h5
Epoch 46/150
238/238 [==============================] - 55s 229ms/step - loss: 1.4732 - fbeta_score: 0.5845

Epoch 00046: fbeta_score improved from 0.57346 to 0.57728, saving model to CNN_BiLSTM.h5
Epoch 47/150
238/238 [=

238/238 [==============================] - 54s 227ms/step - loss: 1.0412 - fbeta_score: 0.6827

Epoch 00085: fbeta_score did not improve from 0.67531
Epoch 86/150
238/238 [==============================] - 54s 228ms/step - loss: 1.0311 - fbeta_score: 0.6830

Epoch 00086: fbeta_score did not improve from 0.67531
Epoch 87/150
238/238 [==============================] - 55s 229ms/step - loss: 1.0048 - fbeta_score: 0.6934

Epoch 00087: fbeta_score improved from 0.67531 to 0.67943, saving model to CNN_BiLSTM.h5
Epoch 88/150
238/238 [==============================] - 54s 226ms/step - loss: 1.0118 - fbeta_score: 0.6890

Epoch 00088: fbeta_score improved from 0.67943 to 0.68440, saving model to CNN_BiLSTM.h5
Epoch 89/150
238/238 [==============================] - 55s 231ms/step - loss: 1.0111 - fbeta_score: 0.6916

Epoch 00089: fbeta_score improved from 0.68440 to 0.68476, saving model to CNN_BiLSTM.h5
Epoch 90/150
238/238 [==============================] - 55s 230ms/step - loss: 0.9984 - fbeta

Epoch 130/150
238/238 [==============================] - 55s 231ms/step - loss: 0.8412 - fbeta_score: 0.7368

Epoch 00130: fbeta_score improved from 0.72976 to 0.73312, saving model to CNN_BiLSTM.h5
Epoch 131/150
238/238 [==============================] - 55s 230ms/step - loss: 0.8085 - fbeta_score: 0.7440

Epoch 00131: fbeta_score improved from 0.73312 to 0.73602, saving model to CNN_BiLSTM.h5
Epoch 132/150
238/238 [==============================] - 55s 231ms/step - loss: 0.8236 - fbeta_score: 0.7376

Epoch 00132: fbeta_score did not improve from 0.73602
Epoch 133/150
238/238 [==============================] - 55s 231ms/step - loss: 0.8206 - fbeta_score: 0.7412

Epoch 00133: fbeta_score did not improve from 0.73602
Epoch 134/150
238/238 [==============================] - 55s 230ms/step - loss: 0.8118 - fbeta_score: 0.7440

Epoch 00134: fbeta_score did not improve from 0.73602
Epoch 135/150
238/238 [==============================] - 55s 231ms/step - loss: 0.7929 - fbeta_score: 0.7508

